# Calculate robusteness from pi control runs

In [31]:
# Some definitions

# directory of data
#fdir = '~/Google Drive/professional/research/FARALLON_INSTITUTE_PROJECTS/2020 NOAA MAPP/Climate_extremes_sharedfigsandcode/data/FOR SCATTER PLOT/'
fdir = '/Volumes/GoogleDrive/My Drive/Climate_extremes_sharedfigsandcode/data/Annual_TimeSeries/'

In [32]:
# Modules
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter('ignore') #filter some warning messages
import seaborn as sns

In [33]:
#detrend function: detrend use polynomial fit
def detrend(x,y,degree):
    import statsmodels.formula.api as smf
    import numpy as np
    import pandas as pd
    

    df = pd.DataFrame(columns=['y', 'x'])
    df['x'] = x
    df['y'] = pd.Series(y)   #y

    idx = np.isfinite(x) & np.isfinite(y)
    weights = np.polyfit(x[idx], y[idx], degree)
    model = np.poly1d(weights)
    results = smf.ols(formula='y ~ model(x)', data=df).fit()

    p_value=results.f_pvalue
    r2=results.rsquared_adj

    detrend_ts=np.zeros(shape=(len(y),))
    detrend_ts[:] = np.nan
    detrend_ts[idx]=y[idx]-results.fittedvalues

    #plt.figure(figsize=(10,4))
    #plt.plot(x,y)
    #plt.plot(x[idx], results.fittedvalues, 'r')
    #plt.figure(figsize=(10,4))
    #plt.plot(x, detrend_ts)
    #plt.show()

    return p_value, r2, detrend_ts

In [34]:
def num_coevents(y1, y2, pct1, pct2, runs='picontrol',loc='cclme'):
    # 
    fin1 = fdir+'original ts/'+runs+'_'+loc+'_sst.nc'
    fin2 = fdir+'original ts/'+runs+'_sm.nc'
    modsst = xr.open_dataset(fin1)
    modsst.close()
    modsm2 = xr.open_dataset(fin2)
    modsm2.close()
    
    models = modsm2.model

    # selec periood
    modsst = modsm2.sel(year=slice(y1,y2))
    modsm2 = modsm2.sel(year=slice(y1-1,y2-1))

    nev = list()
    mhw = list()
    drt = list()    
    for ix,i in enumerate(models):
        #print(ix,i)
        tmp1_0= modsst.sel(model=i).sm.values
        tmp2_0= modsm2.sel(model=i).sm.values
        
        #detrend
        nsample = np.linspace(1,len(tmp1_0), len(tmp1_0))
        [p_value, r2, tmp1]= detrend(nsample,tmp1_0,1)
        nsample = np.linspace(1,len(tmp2_0), len(tmp2_0))
        [p_value, r2, tmp2]= detrend(nsample,tmp2_0,1)
        
        # calculate threshold
        mhw_thr = np.nanpercentile(tmp1,pct1)
        drg_thr = np.nanpercentile(tmp2,pct2)
    
        a1 = tmp1<=drg_thr
        a2 = tmp2<=drg_thr
        
        tmp = np.full((len(modsst.year.values),1),1)
        tmp = tmp[a1*a2]
        
        tmp1 = np.full((len(modsst.year.values),1),1)
        tmp1 = tmp1[a1]
        tmp2 = np.full((len(modsst.year.values),1),1)
        tmp2 = tmp2[a2]
        
        nev.append(len(tmp)/30*100) #/(y2-y1+1)) #freq.
        mhw.append(len(tmp1)/30*100)
        drt.append(len(tmp2)/30*100)        
    return nev , mhw, drt # sum(nev)

In [35]:
ny = 30
prc1 = 90
prc2 = 10
loc='cclme'

pinev=list()
pim=list()
pid=list()
for i in range(int(500/ny)):
    nev,m0,d0 = num_coevents(i*ny+2, (i+1)*ny+1, prc1 , prc2,loc=loc)
    pinev.append(nev)
    pim.append(m0)
    pid.append(d0)
pinev,np.mean(pinev) ,np.mean(pim), np.mean(pid) 

([[3.3333333333333335,
   0.0,
   3.3333333333333335,
   3.3333333333333335,
   0.0,
   3.3333333333333335,
   0.0,
   0.0,
   6.666666666666667,
   0.0,
   3.3333333333333335,
   6.666666666666667,
   3.3333333333333335,
   0.0,
   10.0,
   0.0,
   3.3333333333333335,
   0.0,
   0.0,
   3.3333333333333335,
   0.0,
   0.0],
  [6.666666666666667,
   0.0,
   0.0,
   0.0,
   3.3333333333333335,
   6.666666666666667,
   0.0,
   3.3333333333333335,
   3.3333333333333335,
   6.666666666666667,
   3.3333333333333335,
   3.3333333333333335,
   3.3333333333333335,
   0.0,
   3.3333333333333335,
   3.3333333333333335,
   3.3333333333333335,
   3.3333333333333335,
   0.0,
   3.3333333333333335,
   6.666666666666667,
   3.3333333333333335],
  [6.666666666666667,
   3.3333333333333335,
   0.0,
   0.0,
   3.3333333333333335,
   0.0,
   0.0,
   0.0,
   3.3333333333333335,
   3.3333333333333335,
   0.0,
   3.3333333333333335,
   3.3333333333333335,
   0.0,
   6.666666666666667,
   0.0,
   6.6666666666

In [36]:
#model by model

In [37]:
model_mean=np.mean(pinev,axis=0)
model_spread=np.std(pinev,axis=0)
IV=(np.sum((model_spread*model_spread))/22)**.5
IV,model_spread

(2.1550167568309586,
 array([2.3570226 , 2.49130432, 2.03058216, 2.03058216, 2.27264836,
        2.31990182, 1.44337567, 1.65359457, 2.31990182, 2.49130432,
        2.11435241, 2.19492787, 1.44337567, 1.54504135, 2.34779743,
        2.42061459, 1.86338998, 2.34779743, 2.6020825 , 1.99826313,
        2.49130432, 2.03058216]))

In [38]:
#16 30-yr periods MMEs

In [39]:
mme_mean=np.mean(pinev,axis=1)
mme_spread=np.std(pinev,axis=1)
#internal variability mean and spread
mme_mean,np.mean(mme_spread)

(array([2.27272727, 3.03030303, 2.27272727, 2.57575758, 3.03030303,
        2.42424242, 2.57575758, 2.72727273, 2.27272727, 2.42424242,
        2.42424242, 2.42424242, 2.12121212, 2.87878788, 2.27272727,
        3.48484848]),
 2.332895442724399)

In [40]:
#MME mean and spread

In [41]:
mme_model_mean=np.mean(mme_mean)
mme_model_spread=np.std(mme_mean)
mme_model_mean,mme_model_spread

(2.575757575757576, 0.3553345272593507)

In [42]:
#cross model mean and spread

In [43]:
cross_model_mean=np.mean(model_mean)
cross_model_spread=np.std(model_mean)
cross_model_mean,cross_model_spread

(2.5757575757575757, 0.984302004559567)

In [44]:
IV,cross_model_spread,mme_model_spread,np.std(pinev),np.std(pinev)-IV

(2.1550167568309586,
 0.984302004559567,
 0.3553345272593507,
 2.3691660259260443,
 0.2141492690950857)

In [45]:
## Calculate thresholds for PI Control runs

In [46]:
def get_thre(loc,y1,y2,prc1,prc2):
    # open data: anomalies (without trend)

    modsst = xr.open_dataset(fdir+'original ts/picontrol_'+loc+'_sst.nc')
    modsst.close()
    modsm2 = xr.open_dataset(fdir+'original ts/picontrol_sm.nc')
    modsm2.close()

    # select period 
    modsst = modsm2.sel(year=slice(y1,y2))
    modsm2 = modsm2.sel(year=slice(y1-1,y2-1))
    mhw_thr = list()
    drg_thr = list()
    models = list()

    # Calculate thresholds for each model
    for ix,i in enumerate(modsm2.model.values):
        tmp1_0= modsst.sel(model=i).sm.values
        tmp2_0= modsm2.sel(model=i).sm.values
        
        #detrend
        nsample = np.linspace(1,len(tmp1_0), len(tmp1_0))
        [p_value, r2, tmp1]= detrend(nsample,tmp1_0,1)
        nsample = np.linspace(1,len(tmp2_0), len(tmp2_0))
        [p_value, r2, tmp2]= detrend(nsample,tmp2_0,1)
        
        mhw_thr.append(np.nanpercentile(tmp1,prc1))
        drg_thr.append(np.nanpercentile(tmp2,prc2))

        models.append(i)
    
    return mhw_thr, drg_thr , models

In [47]:
# calculate number of events for control, historical and future
# based on pi control thresholds

def freq_coevents(fdir, runs, loc, mhw_thr, drg_thr, models, y1, y2):
    #pi control
    fi1 = fdir+'original ts/'+runs+'_'+loc+'_sst.nc'
    fi2 = fdir+'original ts/'+runs+'_sm.nc'
    modsst = xr.open_dataset(fi1)
    modsst.close()
    modsm2 = xr.open_dataset(fi2)
    modsm2.close()
    models=modsm2.model

    # select period
    modsst = modsm2.sel(year=slice(y1,y2))
    modsm2 = modsm2.sel(year=slice(y1-1,y2-1))

    nev = list()
    drt=list()
    mhw=list()
    for ix,i in enumerate(models):
        #print(ix,i)
        tmp1_0= modsst.sel(model=i).sm.values
        tmp2_0= modsm2.sel(model=i).sm.values
        
        #detrend
        nsample = np.linspace(1,len(tmp1_0), len(tmp1_0))
        [p_value, r2, tmp1]= detrend(nsample,tmp1_0,1)
        nsample = np.linspace(1,len(tmp2_0), len(tmp2_0))
        [p_value, r2, tmp2]= detrend(nsample,tmp2_0,1)
        
        a1 = tmp1<=drg_thr[ix]
        a2 = tmp2<=drg_thr[ix]
        tmp = np.full((len(modsst.year.values),1),1)
        tmp = tmp[a1*a2]
        
        tmp1 = np.full((len(modsst.year.values),1),1)
        tmp1 = tmp1[a1]
        tmp2 = np.full((len(modsst.year.values),1),1)
        tmp2 = tmp2[a2]
        
        nev.append(len(tmp)/30*100)
        mhw.append(len(tmp1)/30*100)
        drt.append(len(tmp2)/30*100)
    
    freq = np.array(nev) #/(y2-y1+1)
    
    return nev,  mhw, drt #mean no. of events
#freq, np.round(np.nanmean(freq),4), np.round(np.nanstd(freq),4) #mean frequency


In [48]:
def freq_difruns(loc,ny, y0,y,prc1, prc2):
    
    # calculate threshold in pi control
    #y2=2014
    #y1=y2-ny+1
    mhw_thr, drg_thr, models = get_thre(loc,y0,y,prc1,prc2)

    print ('Percentiles: '+str(prc1)+'/'+str(prc2))
    # pi control
    print('piControl: '+str(y0)+'-'+str(y))
    y2=31
    y1=2
    freq0, mfrq, sfrq = freq_coevents(fdir,'picontrol',loc, mhw_thr,drg_thr, models,y0,y)
    print(mfrq,sfrq)

    # historical 1
    y1 = 1901
    y2 = y1+ny-1
    print('\nhistorical: '+str(y1)+'-'+str(y2))
    freq, mfrq, sfrq = freq_coevents(fdir,'historical',loc,mhw_thr,drg_thr, models, y1,y2)
    print(mfrq,sfrq)

    # historical 2
    y2=2029
    y1=y2-ny+1
    print('\nhistorical 2: '+str(y1)+'-'+str(y2))
    freq1, mfrq1, sfrq1 = freq_coevents(fdir,'histssp585',loc,mhw_thr,drg_thr, models, y1,y2)
    print(mfrq1,sfrq1)


    # historical 2
    y2=2014
    y1=y2-ny+1
    print('\nhistorical 2: '+str(y1)+'-'+str(y2))
    freq, mfrq, sfrq = freq_coevents(fdir,'historical',loc,mhw_thr,drg_thr, models, y1,y2)
    print(mfrq,sfrq)

    # future 1
    y2=2099
    y1=y2-ny+1
    print('\nfuture: '+str(y1)+'-'+str(y2))
    freq2, mfrq2, sfrq2 = freq_coevents(fdir,'ssp585',loc,mhw_thr,drg_thr, models, y1,y2)
    print(mfrq2,sfrq2)
    
    # future 2
    y2=2069
    y1=y2-ny+1
    print('\nfuture: '+str(y1)+'-'+str(y2))
    freq, mfrq, sfrq = freq_coevents(fdir,'ssp585',loc,mhw_thr,drg_thr, models, y1,y2)
    print(mfrq,sfrq)

    # future 2
    y2=2045+20
    y1=y2-ny+1
    print('\nfuture: '+str(y1)+'-'+str(y2))
    freq, mfrq, sfrq = freq_coevents(fdir,'ssp585',loc,mhw_thr,drg_thr, models, y1,y2)
    print(mfrq,sfrq)
    
    return freq0,freq1,freq2,mfrq1, sfrq1,mfrq2, sfrq2

In [49]:
# Frequency of events - detrended data, using pi control threshols
## percentiles 90-10

In [50]:

print ('\nPercentiles: '+str(prc1)+'/'+str(prc2))
print('\nProbable # events in '+str(ny)+' years'+': '+str(ny*0.01))
funev=list()
hinev=list()
test=list()
him=list()
hid=list()
fum=list()
fud=list()
for i in range(int(500/ny)):
    pi,hi,fu,m1,d1,m2,d2 = freq_difruns(loc,ny,i*ny+2, (i+1)*ny+1, prc1, prc2)
    test.append(pi)
    funev.append(fu)
    hinev.append(hi)
    him.append(m1)
    hid.append(d1)
    fum.append(m2)
    fud.append(d2)
np.mean(funev),np.mean(hinev),np.mean(him),np.mean(hid),np.mean(fum),np.mean(fud)


Percentiles: 90/10

Probable # events in 30 years: 0.3
Percentiles: 90/10
piControl: 2-31
[13.333333333333334, 10.0, 13.333333333333334, 13.333333333333334, 10.0, 13.333333333333334, 13.333333333333334, 13.333333333333334, 16.666666666666664, 3.3333333333333335, 6.666666666666667, 13.333333333333334, 10.0, 13.333333333333334, 13.333333333333334, 6.666666666666667, 6.666666666666667, 10.0, 10.0, 6.666666666666667, 10.0, 10.0] [10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0]

historical: 1901-1930
[13.333333333333334, 6.666666666666667, 13.333333333333334, 13.333333333333334, 30.0, 33.33333333333333, 16.666666666666664, 16.666666666666664, 20.0, 0.0, 20.0, 3.3333333333333335, 23.333333333333332, 23.333333333333332, 6.666666666666667, 13.333333333333334, 6.666666666666667, 20.0, 6.666666666666667, 10.0, 16.666666666666664, 10.0] [13.333333333333334, 6.666666666666667, 13.333333333333334, 10.0, 26.66666666

[0.0, 20.0, 3.3333333333333335, 20.0, 23.333333333333332, 13.333333333333334, 6.666666666666667, 10.0, 13.333333333333334, 13.333333333333334, 13.333333333333334, 3.3333333333333335, 10.0, 6.666666666666667, 26.666666666666668, 10.0, 0.0, 10.0, 6.666666666666667, 23.333333333333332, 23.333333333333332, 13.333333333333334] [0.0, 23.333333333333332, 3.3333333333333335, 20.0, 20.0, 13.333333333333334, 6.666666666666667, 6.666666666666667, 16.666666666666664, 10.0, 16.666666666666664, 3.3333333333333335, 10.0, 6.666666666666667, 20.0, 10.0, 3.3333333333333335, 6.666666666666667, 6.666666666666667, 23.333333333333332, 26.666666666666668, 13.333333333333334]
Percentiles: 90/10
piControl: 62-91
[10.0, 13.333333333333334, 13.333333333333334, 6.666666666666667, 10.0, 10.0, 10.0, 10.0, 10.0, 13.333333333333334, 10.0, 13.333333333333334, 10.0, 13.333333333333334, 10.0, 10.0, 13.333333333333334, 10.0, 13.333333333333334, 10.0, 13.333333333333334, 10.0] [10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10

[16.666666666666664, 13.333333333333334, 3.3333333333333335, 16.666666666666664, 16.666666666666664, 10.0, 10.0, 13.333333333333334, 26.666666666666668, 30.0, 23.333333333333332, 30.0, 16.666666666666664, 10.0, 23.333333333333332, 13.333333333333334, 3.3333333333333335, 16.666666666666664, 20.0, 20.0, 13.333333333333334, 10.0] [13.333333333333334, 13.333333333333334, 10.0, 13.333333333333334, 16.666666666666664, 13.333333333333334, 10.0, 13.333333333333334, 23.333333333333332, 33.33333333333333, 23.333333333333332, 30.0, 20.0, 10.0, 23.333333333333332, 16.666666666666664, 3.3333333333333335, 13.333333333333334, 23.333333333333332, 23.333333333333332, 13.333333333333334, 10.0]

future: 2036-2065
[10.0, 16.666666666666664, 6.666666666666667, 13.333333333333334, 20.0, 13.333333333333334, 13.333333333333334, 13.333333333333334, 23.333333333333332, 30.0, 23.333333333333332, 23.333333333333332, 16.666666666666664, 6.666666666666667, 16.666666666666664, 10.0, 0.0, 20.0, 16.666666666666664, 23

[13.333333333333334, 13.333333333333334, 10.0, 26.666666666666668, 13.333333333333334, 13.333333333333334, 0.0, 23.333333333333332, 33.33333333333333, 16.666666666666664, 0.0, 3.3333333333333335, 13.333333333333334, 3.3333333333333335, 26.666666666666668, 6.666666666666667, 33.33333333333333, 13.333333333333334, 0.0, 10.0, 26.666666666666668, 6.666666666666667] [6.666666666666667, 13.333333333333334, 10.0, 23.333333333333332, 13.333333333333334, 13.333333333333334, 0.0, 23.333333333333332, 30.0, 10.0, 0.0, 0.0, 13.333333333333334, 3.3333333333333335, 26.666666666666668, 6.666666666666667, 30.0, 16.666666666666664, 0.0, 10.0, 23.333333333333332, 6.666666666666667]

future: 2040-2069
[16.666666666666664, 20.0, 13.333333333333334, 23.333333333333332, 10.0, 13.333333333333334, 6.666666666666667, 30.0, 23.333333333333332, 20.0, 10.0, 6.666666666666667, 10.0, 10.0, 30.0, 3.3333333333333335, 10.0, 10.0, 6.666666666666667, 20.0, 26.666666666666668, 10.0] [13.333333333333334, 20.0, 20.0, 23.333

[6.666666666666667, 20.0, 20.0, 10.0, 13.333333333333334, 23.333333333333332, 23.333333333333332, 13.333333333333334, 23.333333333333332, 13.333333333333334, 13.333333333333334, 13.333333333333334, 10.0, 6.666666666666667, 13.333333333333334, 6.666666666666667, 13.333333333333334, 3.3333333333333335, 10.0, 6.666666666666667, 30.0, 20.0] [6.666666666666667, 20.0, 20.0, 6.666666666666667, 13.333333333333334, 26.666666666666668, 30.0, 13.333333333333334, 23.333333333333332, 13.333333333333334, 10.0, 20.0, 10.0, 13.333333333333334, 13.333333333333334, 6.666666666666667, 13.333333333333334, 3.3333333333333335, 10.0, 10.0, 30.0, 20.0]

future: 2070-2099
[10.0, 13.333333333333334, 10.0, 6.666666666666667, 16.666666666666664, 23.333333333333332, 26.666666666666668, 13.333333333333334, 23.333333333333332, 16.666666666666664, 3.3333333333333335, 20.0, 13.333333333333334, 3.3333333333333335, 6.666666666666667, 10.0, 20.0, 6.666666666666667, 3.3333333333333335, 10.0, 16.666666666666664, 16.6666666

[13.333333333333334, 16.666666666666664, 10.0, 13.333333333333334, 10.0, 3.3333333333333335, 6.666666666666667, 13.333333333333334, 20.0, 10.0, 20.0, 3.3333333333333335, 10.0, 23.333333333333332, 13.333333333333334, 0.0, 6.666666666666667, 23.333333333333332, 0.0, 10.0, 23.333333333333332, 26.666666666666668] [13.333333333333334, 16.666666666666664, 10.0, 16.666666666666664, 10.0, 10.0, 6.666666666666667, 13.333333333333334, 20.0, 10.0, 20.0, 3.3333333333333335, 10.0, 26.666666666666668, 13.333333333333334, 0.0, 6.666666666666667, 23.333333333333332, 0.0, 10.0, 20.0, 26.666666666666668]

historical 2: 1985-2014
[10.0, 26.666666666666668, 13.333333333333334, 16.666666666666664, 13.333333333333334, 10.0, 6.666666666666667, 13.333333333333334, 26.666666666666668, 6.666666666666667, 26.666666666666668, 3.3333333333333335, 10.0, 23.333333333333332, 13.333333333333334, 6.666666666666667, 10.0, 13.333333333333334, 0.0, 6.666666666666667, 30.0, 20.0] [16.666666666666664, 23.333333333333332, 13

[6.666666666666667, 20.0, 13.333333333333334, 10.0, 6.666666666666667, 20.0, 16.666666666666664, 16.666666666666664, 13.333333333333334, 0.0, 3.3333333333333335, 3.3333333333333335, 23.333333333333332, 23.333333333333332, 20.0, 26.666666666666668, 6.666666666666667, 13.333333333333334, 6.666666666666667, 10.0, 6.666666666666667, 10.0] [6.666666666666667, 20.0, 13.333333333333334, 10.0, 3.3333333333333335, 20.0, 13.333333333333334, 16.666666666666664, 10.0, 0.0, 0.0, 3.3333333333333335, 23.333333333333332, 20.0, 20.0, 26.666666666666668, 6.666666666666667, 3.3333333333333335, 6.666666666666667, 10.0, 6.666666666666667, 10.0]

historical 2: 2000-2029
[6.666666666666667, 13.333333333333334, 26.666666666666668, 20.0, 10.0, 3.3333333333333335, 13.333333333333334, 16.666666666666664, 10.0, 6.666666666666667, 3.3333333333333335, 3.3333333333333335, 16.666666666666664, 23.333333333333332, 20.0, 13.333333333333334, 6.666666666666667, 23.333333333333332, 6.666666666666667, 13.333333333333334, 20

[6.666666666666667, 16.666666666666664, 20.0, 13.333333333333334, 23.333333333333332, 13.333333333333334, 13.333333333333334, 26.666666666666668, 23.333333333333332, 23.333333333333332, 13.333333333333334, 13.333333333333334, 10.0, 10.0, 26.666666666666668, 23.333333333333332, 16.666666666666664, 6.666666666666667, 20.0, 26.666666666666668, 23.333333333333332, 10.0] [10.0, 23.333333333333332, 16.666666666666664, 13.333333333333334, 20.0, 13.333333333333334, 13.333333333333334, 23.333333333333332, 26.666666666666668, 23.333333333333332, 16.666666666666664, 13.333333333333334, 10.0, 6.666666666666667, 20.0, 30.0, 16.666666666666664, 6.666666666666667, 16.666666666666664, 26.666666666666668, 23.333333333333332, 13.333333333333334]
Percentiles: 90/10
piControl: 392-421
[10.0, 16.666666666666664, 10.0, 10.0, 10.0, 10.0, 13.333333333333334, 16.666666666666664, 13.333333333333334, 13.333333333333334, 10.0, 16.666666666666664, 10.0, 10.0, 10.0, 13.333333333333334, 10.0, 10.0, 13.33333333333333

[6.666666666666667, 13.333333333333334, 26.666666666666668, 16.666666666666664, 6.666666666666667, 16.666666666666664, 10.0, 20.0, 26.666666666666668, 13.333333333333334, 10.0, 30.0, 16.666666666666664, 20.0, 30.0, 13.333333333333334, 33.33333333333333, 6.666666666666667, 6.666666666666667, 26.666666666666668, 13.333333333333334, 13.333333333333334] [6.666666666666667, 13.333333333333334, 26.666666666666668, 13.333333333333334, 6.666666666666667, 16.666666666666664, 10.0, 20.0, 23.333333333333332, 16.666666666666664, 16.666666666666664, 33.33333333333333, 16.666666666666664, 20.0, 30.0, 16.666666666666664, 23.333333333333332, 10.0, 6.666666666666667, 33.33333333333333, 13.333333333333334, 13.333333333333334]

future: 2036-2065
[3.3333333333333335, 16.666666666666664, 23.333333333333332, 13.333333333333334, 3.3333333333333335, 16.666666666666664, 6.666666666666667, 23.333333333333332, 23.333333333333332, 13.333333333333334, 13.333333333333334, 26.666666666666668, 13.333333333333334, 20.

(4.308712121212121,
 5.170454545454546,
 13.058712121212123,
 13.27651515151515,
 13.001893939393938,
 13.087121212121211)

In [51]:
loc,ny

('cclme', 30)

In [52]:
N=int(500/ny)
N

16

In [53]:
np.mean(model_mean),np.mean(hinev),np.mean(funev)

(2.5757575757575757, 5.170454545454546, 4.308712121212121)

In [54]:
#hist - picontrol FAR
diff=list()
for k in range(len(pinev)):
    zip_obj=zip(hinev[k],pinev[k])
    for i , j in zip_obj:
            diff.append(i-j)
    #diff mean, model spread
diff_a=np.asarray(diff).reshape(N,22)
diff_mean=np.mean(diff_a,axis=1)
diff_spread=np.std(diff_mean)
diff_mean.mean(),diff_spread

(2.5946969696969697, 1.0712064956347933)

In [55]:
hinev_a=np.asarray(hinev).reshape(N,22).sum(axis=1)
pinev_a=np.asarray(pinev).reshape(N,22).sum(axis=1)
far=(hinev_a-pinev_a)/hinev_a
FAR=np.mean(far)
sd=np.std(far)
FAR,sd

(0.479545669863114, 0.16289752555768652)

In [56]:
import statsmodels.api as sm
import statsmodels as sm
import pandas
from patsy import dmatrices
cu,cl=sm.stats.proportion.proportion_confint(FAR, 1, alpha=0.05, method='normal') #'binom_test''normal'
FAR,cu,cl

(0.479545669863114, 0.0, 1.0)

In [57]:
#future - hist change
#cross-mme spread
diff=list()
for k in range(len(pinev)):
    zip_obj=zip(funev[k],hinev[k])
    for i , j in zip_obj:
            diff.append(i-j)
    #diff mean, model spread
diff_a=np.asarray(diff).reshape(N,22)
diff_mean=np.mean(diff_a,axis=1)
diff_spread=np.std(diff_mean)
diff_mean.mean(),diff_spread

(-0.8617424242424242, 0.5559768690562763)

In [58]:
#cross-model spread
diff=list()
for k in range(len(pinev)):
    zip_obj=zip(funev[k],hinev[k])
    for i , j in zip_obj:
            diff.append(i-j)
    #diff mean, model spread
diff_mean=np.mean(diff)
diff_spread=np.std(diff)-IV
diff_mean,diff_spread

(-0.8617424242424243, 2.9680717996842567)

In [59]:
funev_a=np.asarray(funev).reshape(N,22).sum(axis=1)
pinev_a=np.asarray(pinev).reshape(N,22).sum(axis=1)
far=(funev_a-pinev_a)/funev_a
FAR=np.mean(far)
sd=np.std(far)
FAR=np.mean(far)
cu,cl=sm.stats.proportion.proportion_confint(FAR, 1, alpha=0.05, method='normal') #'binom_test''normal'
FAR,cu,cl,sd

(0.3821534816066065, 0.0, 1.0, 0.16056370501336426)

In [60]:
#future - pi change
#cross-mme spread
diff=list()
for k in range(len(pinev)):
    zip_obj=zip(funev[k],pinev[k])
    for i , j in zip_obj:
            diff.append(i-j)
    #diff mean, model spread
diff_a=np.asarray(diff).reshape(N,22)
diff_mean=np.mean(diff_a,axis=1)
diff_spread=np.std(diff_mean)
diff_mean.mean(),diff_spread

(1.7329545454545454, 0.8670333742886688)